<a href="https://colab.research.google.com/github/fatemafaria142/Advancing-Agronomic-Question-and-Answering-through-Various-Language-Models/blob/main/Agronomic_Question_and_Answering_using_Starling_LM_7B_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [56]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **Dataset Link:** https://github.com/JonaOmara/AgroQA-Dataset/tree/main

In [57]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/AgroQA Dataset.csv")
dataset.head()

,Crop,Question,Answer
0,maize,"Apart from hand weeding, what other method use...",Machinery weeders are available
1,beans,"Apart from insecticide, what other method used...",Use resistant verities and increase on water a...
2,maize,Apart from sun drying which other method used ...,Use tarpaulins or cemented floor free from dust
3,cassava,"Apart from sun drying, what other method can I...",Solar driers
4,beans,As a farmer when should I harvest beans.,When the beans pods are yellowish green or dry...


In [58]:
length_of_data = len(dataset)
# Print the length
print(length_of_data)

3044


In [59]:
from datasets import Dataset, DatasetDict

# Create a DatasetDict for instruct_tune_dataset
instruct_tune_dataset = DatasetDict()

# Select 1500 rows for the 'train' split and only keep 'Question' and 'Answer'
instruct_tune_dataset["train"] = Dataset.from_pandas(dataset[["Question", "Answer"]].head(2700))

# Select 200 rows for the 'test' split and only keep 'Question' and 'Answer'
instruct_tune_dataset["test"] = Dataset.from_pandas(dataset[["Question", "Answer"]].tail(200))

# Print the updated instruct_tune_dataset
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 2700
    })
    test: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 200
    })
})


### **Prompt Creation**

In [62]:
def create_prompt(sample):
    # Use a predefined template for instructions
    prompt =  "Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. "
    prompt += sample['Question']
    single_turn_prompt = f"GPT4 Correct User: {prompt}<|end_of_turn|>GPT4 Correct Assistant: {sample['Answer']}"
    return single_turn_prompt

In [63]:
create_prompt(instruct_tune_dataset["train"][0])

"GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. Apart from hand weeding, what other method used to weed maize<|end_of_turn|>GPT4 Correct Assistant: Machinery weeders are available"

In [64]:
create_prompt(instruct_tune_dataset["train"][1])

"GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. Apart from insecticide, what other method used to control bean weevils?<|end_of_turn|>GPT4 Correct Assistant: Use resistant verities and increase on water availability for crop vigor"

In [65]:
create_prompt(instruct_tune_dataset["train"][2])

"GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. Apart from sun drying which other method used for drying maize<|end_of_turn|>GPT4 Correct Assistant: Use tarpaulins or cemented floor free from dust"

In [66]:
create_prompt(instruct_tune_dataset["train"][3])

"GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. Apart from sun drying, what other method can I use to dry cassava<|end_of_turn|>GPT4 Correct Assistant: Solar driers"

In [67]:
create_prompt(instruct_tune_dataset["train"][4])

"GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. As a farmer when should I harvest beans.<|end_of_turn|>GPT4 Correct Assistant: When the beans pods are yellowish green or dry brown"

In [68]:
create_prompt(instruct_tune_dataset["train"][5])

"GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. As a farmer which method is good to prevent soil erosion<|end_of_turn|>GPT4 Correct Assistant: Farmers can terrace their gardens or mulch their gardens to stop erosion"

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [15]:
mode_id = "berkeley-nest/Starling-LM-7B-alpha"

In [16]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained("berkeley-nest/Starling-LM-7B-alpha")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [69]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [70]:
prompt = """GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. As a farmer which method is good to prevent soil erosion? <|end_of_turn|>GPT4 Correct Assistant:"""

In [71]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


"<s> GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. As a farmer which method is good to prevent soil erosion? <|end_of_turn|> GPT4 Correct Assistant: As a farmer, one method to prevent soil erosion is by installing and maintaining terraces, contour lines, and vegetation barriers. These methods help to reduce the rate at which soil is lost through wind and water erosion, preserving nutrients and fertile topsoil.<|end_of_turn|>"

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [72]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [73]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [74]:
from transformers import TrainingArguments
output_model= "Starling_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [75]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [76]:
trainer.train()

Step,Training Loss
10,1.306100
20,0.854900
30,0.618000
40,0.576100
50,0.520400
60,0.536500
70,0.530000
80,0.520900
90,0.465400
100,0.538600


TrainOutput(global_step=100, training_loss=0.6467003774642944, metrics={'train_runtime': 531.4844, 'train_samples_per_second': 0.753, 'train_steps_per_second': 0.188, 'total_flos': 4370888392704000.0, 'train_loss': 0.6467003774642944, 'epoch': 0.4})

### **Save the model**

In [77]:
trainer.save_model("Starling_instruct_generation")

In [78]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [79]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [84]:
prompt = "GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. "
prompt += "What causes premature stem cracks in maize? <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. What causes premature stem cracks in maize? <|end_of_turn|>  GPT4 Correct Assistant: Premature stem cracks in maize can be caused by a variety of factors, such as drought, irregular watering, high temperatures, and improper soil preparation. Cracks can also be caused by various diseases and pests, such as root rot and fungal infections that compromise the plant's structure. Additionally, genetic factors can also contribute to premature stem cracks in maize.<|end_of_turn|>


### **Example No:2**

In [85]:
prompt = "GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. "
prompt += "Which variety of cassava is tolerant to disease? <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. Which variety of cassava is tolerant to disease? <|end_of_turn|>  GPT4 Correct Assistant: Cassava varieties that are developed are resistant to disease such as Cassava Mosaic Virus, Cassava Brown Streak Virus and Cassava bacterial blight among others.<|end_of_turn|>


### **Example No:3**

In [86]:
prompt = "GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. "
prompt += "can rain water reduce power of chemical after spraying?  <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. can rain water reduce power of chemical after spraying?  <|end_of_turn|>  GPT4 Correct Assistant: No, Rainwater does not reduce the power of the chemicals after they've been sprayed.<|end_of_turn|>


### **Example No:4**

In [87]:
prompt = "GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. "
prompt += "What is the advantage of growing Maize? <|end_of_turn|> "
prompt += "GPT4 Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> GPT4 Correct User: Imagine you're an experienced agronomist, entrusted with optimizing crop yields. Your expertise is crucial in guiding farmers towards success. Picture a field filled with potential, and now it's your task to provide the necessary insights for cultivating a thriving harvest. What is the advantage of growing Maize? <|end_of_turn|>  GPT4 Correct Assistant: The advantage of growing maize includes its high yield potential, nutritive benefits, ability to grow on a wide range of soils, and its role in contributing to food security and income generation for farmers.<|end_of_turn|>
